In [1]:
import torch            
from torch import nn
import torch.nn.functional as F
import pickle
import numpy as np
import jieba

In [2]:
def preprocess_data(data):
    #实现，每一个intent下面有多少个同义句
    intentlist = []#所有的intent集合
    for item in data:
        if not item['intent'] in intentlist:
            intentlist.append(item['intent'])
            
    textlist = [0 for i in range(len(intentlist))]
    for i in range(len(data)):
        if data[i]['intent'] in intentlist:
            textlist[intentlist.index(data[i]['intent'])] += 1 
    
    #把同义句低于5个的intent包含部分删除，intentlist包含的是所有的intent集合，textlist：intentlist对应项的同义句个数
    del_data = []
    for item in data:
        if (textlist[intentlist.index(item['intent'])] >4):
            del_data.append(item)
        else:
            textlist[intentlist.index(item['intent'])]  = 0
    print(len(del_data))
    return del_data, intentlist

In [3]:
def generate_id(data):   
    intent2id = {}
    id2intent = {}
    for item in data:
        if not item['intent'] in intent2id:
            len_intent2id = len(intent2id)
            intent2id[item['intent']] = len_intent2id
            id2intent[len_intent2id] = item['intent']
    with open("intent2id",'wb') as f:
            pickle.dump(intent2id,f) 
    with open("id2intent",'wb') as f:
            pickle.dump(id2intent,f) 

    word2id={}
    word2id['PAD'] = 0
    word2id['UNK'] = 1
    id2word={}
    id2word[0] = 'PAD'
    id2word[1] = 'UNK'
    for item in data:
        seg_text = jieba.cut(item['text'],cut_all=False)
        seg_text = (list(seg_text))   
        for w in seg_text:
            if w not in word2id:
                len_word2id = len(word2id) 
                word2id[w] = len_word2id
                id2word[len_word2id] = w
        item['text']=seg_text#使用结巴分词已经划分好
    with open("word2id",'wb') as f:
            pickle.dump(word2id,f) 
    with open("id2word",'wb') as f:
            pickle.dump(id2word,f)

In [4]:
def load_pretrained_word2vec(emb_size):
    with open('word2id','rb') as f:
        word2id = pickle.load(f)  #2889个
    f.close()
    weights = np.zeros([len(word2id),emb_size], dtype=np.float32)
    
    word2vec = {}
    lines_num = 0
    with open('financial_bigram-char',errors='ignore') as f:
        first_line = True
        for line in f:
            if first_line:
                first_line = False
                dim = int(line.rstrip().split()[1])
                continue
            lines_num += 1
            tokens = line.rstrip().split(' ')
            word2vec[tokens[0]] = np.asarray([float(x) for x in tokens[1:]])

    for word_i in word2id:
        if word_i in word2vec:
            weights[word2id[word_i]] = word2vec[word_i]
        else:
            weights[word2id[word_i]] = np.random.uniform(-0.1,0.1)
    weights = torch.from_numpy(weights)
    print(weights.shape)
    with open('renmin_emb_weights','wb') as g:
        pickle.dump(weights,g) 
    return weights
#load_pretrained_word2vec(300)
    

In [5]:
import itertools
import numpy as np

def sample_sq(intent2text,intent_texts_len,sample_classes,way,shot,query):
    
    
    support_idx, query_idx = {},{}

    for class_i in sample_classes: 
        sample_set = []
        sample_set = intent2text[str(class_i)]#二维数组.intent对应的所有text
        text_content,text_lens ,sample_s_nums = [],[],[]
        
        #sample_s_nums找出intent2text某一个intent（sample_set）下面，数组选哪几个位置的text，位置集合
        sample_s_nums = np.random.permutation(len(sample_set))[:shot] 
        text_content = [sample_set[item] for item in sample_s_nums]
        text_lens = [intent_texts_len[str(class_i)][num] for num in sample_s_nums]
        support_idx[str(class_i)] = {'text_content':text_content,'text_len':text_lens}
       
        #query_set构建
        text_content,text_lens ,sample_query_nums,sample_q_nums = [],[],[],[]
        sample_query_nums = [i for i in range(len(sample_set)) if i not in sample_s_nums]#集合减小
        sample_q_nums = np.random.permutation(sample_query_nums)[:query]
        text_content = [sample_set[item] for item in sample_q_nums]
        text_lens = [intent_texts_len[str(class_i)][num] for num in sample_q_nums]
        query_idx[str(class_i)] = {'text_content':text_content,'text_len':text_lens}
    return support_idx, query_idx

In [6]:
def dict_2_2array(x):
    sq_idx = x
    
    sq_ways_text = None #把所有classes的text集合成一个二维数组
    for i in sq_idx:
        if sq_ways_text==None:
            sq_ways_text = sq_idx[i]['text_content'] 
        else:
            sq_ways_text.extend(sq_idx[i]['text_content'])
      
    
    sq_ways_text_lens = None
    for i in sq_idx:
        if sq_ways_text_lens==None:
            sq_ways_text_lens = sq_idx[i]['text_len'] 
        else:
            sq_ways_text_lens.extend(sq_idx[i]['text_len'])
    
    sq_ways_text = torch.tensor(sq_ways_text)
    sq_ways_text_lens = torch.tensor(sq_ways_text_lens)
    #print(sq_ways_text.size())
    return sq_ways_text,sq_ways_text_lens

In [7]:
#构建最终数据,生成传入模型的intent2text，intent_texts_len
def generate_intent2text_intent_texts_len(data):
    intent2text = {}
    with open('id2word','rb') as f:
        id2word = pickle.load(f)
    with open('id2intent','rb') as f:
        id2intent = pickle.load(f)
    for item in (data):
        i = item['intent']
        j = item['text']
        if str(i) in intent2text:
            intent2text[str(i)].append(j)
        else:
            intent2text[str(i)] = [j]    
        
   #重复某些句子以扩充到30个，多的删减到100个
    for item in intent2text: 
        if len(intent2text[item])<30:#
            l_past = len(intent2text[item])
            a = intent2text[item]
            extra = np.random.permutation(a)[:(30-l_past)].tolist()
            intent2text[item].extend(extra)
        elif len(intent2text[item])>100:#
            l_past = len(intent2text[item])
            a = intent2text[item]
            intent2text[item] = np.random.permutation(a)[:100].tolist()   

    intent_texts_len = {}#生成句子对应的长度
    for item in intent2text:
        texts = intent2text[item]
        for text_i in texts:
            if 0 in text_i: index_0 = text_i.index(0)
            else: index_0 = 19#找到长度
            if str(item) in intent_texts_len:
                intent_texts_len[str(item)].append(index_0+1)
            else:
                intent_texts_len[str(item)] = [(index_0+1)]          
    return intent2text, intent_texts_len


In [8]:
def generate_support_query(intent2text, intent_texts_len, sample_classes, way, shot, query):
    support_idx, query_idx = {},{}
    support_ways_text,support_ways_text_lens = [],[]
    query_ways_text,query_ways_text_lens = [],[]
    support_idx, query_idx = sample_sq(intent2text,intent_texts_len,sample_classes, way,shot,query)  
    support_ways_text,support_ways_text_lens = dict_2_2array(support_idx)
    query_ways_text,query_ways_text_lens = dict_2_2array(query_idx)
    support_ways_text = support_ways_text.cuda()
    support_ways_text_lens = support_ways_text_lens#.cuda()
    query_ways_text = query_ways_text.cuda()
    query_ways_text_lens = query_ways_text_lens#.cuda()
    return support_ways_text,support_ways_text_lens,query_ways_text,query_ways_text_lens

In [9]:

def init_gru(gru):
    gru.reset_parameters()
    for _, hh, _, _ in gru.all_weights:
        for i in range(0, hh.size(0), gru.hidden_size):
            nn.init.orthogonal_(hh[i:i + gru.hidden_size], gain=1)



#实现model的三个环节
class Encoder(nn.Module):
    def __init__(self, input_size, embed_size, hidden_size, n_layers, weight):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embed_size = embed_size
        self.n_layers = n_layers
        #self.dropout = dropout
        self.embedding = nn.Embedding.from_pretrained(weight, freeze=False)
        self.gru = nn.GRU(embed_size, hidden_size, n_layers, bidirectional=True)
        init_gru(self.gru)

    def forward(self, input_seqs, input_lens, hidden=None):
        """
        forward procedure. No need for inputs to be sorted
        :param input_seqs: Variable of [T,B]
        :param hidden:
        :param input_lens: *numpy array* of len for each input sequence
        :return:
        """      
        batch_size = input_seqs.size(1)
        embedded = self.embedding(input_seqs)
        embedded = embedded.transpose(0, 1)  # [B,T,E]
      
        sort_idx = np.argsort(-input_lens)
        #DELETE = cuda_(torch
        unsort_idx = (torch.LongTensor(np.argsort(sort_idx))).cuda()
        input_lens = input_lens[sort_idx]
        #DELETE = cuda_(torch
        sort_idx = (torch.LongTensor(sort_idx)).cuda()
        embedded = embedded[sort_idx].transpose(0, 1)  # [T,B,E]
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lens)
        outputs, hidden = self.gru(packed, hidden)
        outputs, _ = torch.nn.utils.rnn.pad_packed_sequence(outputs)
        #print(outputs.size(),'encoder')
        outputs = outputs.transpose(0, 1)[unsort_idx].transpose(0, 1).contiguous()
        hidden = hidden.transpose(0, 1)[unsort_idx].transpose(0, 1).contiguous()
        return outputs, hidden, embedded

In [10]:
import torch.nn.functional as F
import torch.nn as nn
class Attn(nn.Module):
    def __init__(self,hidden_size,d_a):
        super(Attn, self).__init__()
        self.hidden_size = hidden_size
        self.d_a = d_a
        self.W2_weights = nn.Parameter(torch.Tensor(1, self.d_a))
        torch.nn.init.xavier_uniform(self.W2_weights.data)

        self.W1 = (nn.Linear(self.hidden_size*2,self.d_a))
        
    def forward(self,encoder_outputs,output_batch):
        output = encoder_outputs
        #encoder_outputs--[length,batch,hidden*2]      
        W1 = torch.tanh(self.W1(output)) #[length,batch,da]
        W1 = W1.permute(1,0,2)#[batch,length,da]
        score = torch.bmm(W1,
                            self.W2_weights  # (1, da)
                            .permute(1, 0)  # (da, 1)
                            .unsqueeze(0)  # (1, da, 1)
                            .repeat(output_batch, 1, 1)
                            # (batch_size, da, 1)
                            )
        score = F.softmax(F.relu(score.squeeze()))
        weighted = torch.mul(output.permute(1,0,2), score.unsqueeze(-1).expand_as(output.permute(1,0,2)))
        representations = weighted.sum(1).squeeze()
        return representations

        

In [11]:
import torch.nn.functional as F
import torch.nn as nn
class Dynamic_route(nn.Module):
    def __init__(self,n_iter,hidden_size):
        super(Dynamic_route,self).__init__()
        self.n_iter = n_iter
        self.embedding = 2*hidden_size
        self.trans_W = nn.Linear(self.embedding,self.embedding)
        
    def forward(self, x):
        x = self.trans_W(x)#[way,shot,vector_embedding]
        logits = torch.zeros(x.size(0),x.size(1),1).cuda()                                                
        for i in range(self.n_iter):
            probs = F.softmax(logits,dim=1)
            #print(probs.size())
            probs_mul_x = torch.mul(x, probs.expand_as(x))
            y = self.squash(probs_mul_x.sum(dim=1,keepdim=True))##[way,1,vector_embedding]
            if i!= self.n_iter-1:
                delta_logits = (y * x).sum(dim=-1, keepdim=True)
                logits = logits + delta_logits
        return y  
        
    def squash(self, tensor, dim=-1):
        squared_norm = (tensor ** 2).sum(dim=dim, keepdim=True)
        scale = squared_norm / (1 + squared_norm)
        return scale * tensor / torch.sqrt(squared_norm)
        

In [12]:
import torch.nn.functional as F
import torch.nn as nn
class Relation_score(nn.Module):
    def __init__(self,k, batch, in_channel, out_channel):
        super(Relation_score,self).__init__()
        self.neural_tensor_net = nn.Bilinear(in_channel,out_channel,k)
        self.mlp = (nn.Linear(k,1))
    def forward(self,query,classes):
        
        relation_vector = self.neural_tensor_net(query,classes)
        final_score = F.sigmoid(self.mlp(relation_vector))
        #print(relation_vector.size())       
        return final_score


In [13]:
class Fewshot(nn.Module):
    def __init__(self, num_words, embed_size, hidden_size, emb_weight, way, shot, query, 
                 dynamic_iter,k, d_a, **kwargs):
        super(Fewshot, self).__init__()
        self.input_size = num_words
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.emb_weights = emb_weight
        self.way = way
        self.shot = shot
        self.query = query
        self.k = k
        self.encoder = Encoder(self.input_size,self.embed_size,self.hidden_size,weight=self.emb_weights,n_layers=1)
        self.att = Attn(self.hidden_size, d_a)
        self.dynamic_route = Dynamic_route(dynamic_iter,self.hidden_size)
        self.relation_score = Relation_score(k, way*query, 2*self.hidden_size, 2*self.hidden_size)
        
    def forward(self, support_text,support_text_len,query_text,query_text_len,train_i):
        output,hidden,_ = self.encoder(support_text,support_text_len)
        weighted_rep = self.att(output,output.size(1))
        C_K = weighted_rep.view(self.way, self.shot, self.hidden_size * 2)
        Classes_vectors = self.dynamic_route(C_K)
        
        output,hidden,_ = self.encoder(query_text,query_text_len)
        query_vectors = self.att(output,output.size(1))
        ex = Classes_vectors
        #print(Classes_vectors.size(),'Classes_vectors')#[5, 1, 256]
        
        loss = torch.tensor(0.0).cuda()
        for i in range(self.way):
            groud_truth = [0.0 for i in range(self.way * self.query)]
            groud_truth[self.query * i : self.query * (i+1)] = [1.0 for i in range(self.query)]
            groud_truth = torch.tensor(groud_truth).cuda()
            #print(query_vectors.size())#([50, 256])
            
            score = self.relation_score(query_vectors,ex[i].repeat(self.way*self.query,1)).squeeze()#type,tensor
            #if train_i%10000==0:
                #print(score[0:20],'-----------',score[20:40],'----------',score[40:60],'----------',
                     #score[60:80],'-----------',score[80:100])
                #print('#############')
            #print(score.size())[50]
            loss += ((score - groud_truth)**2).sum()
                
        return loss
    
    def get_classes_vectors(self, support_text,support_text_len):#注意输入的时候 t.()
        output,hidden,_ = self.encoder(support_text,support_text_len)
        weighted_rep = self.att(output,output.size(1))
        C_K = weighted_rep.view(int(float(support_text_len.size(0))/self.shot), self.shot, self.hidden_size * 2)
        Classes_vectors = self.dynamic_route(C_K)
        return Classes_vectors
    
    def get_classify_result(self,Classes_vectors,query_text,query_text_len):
        with open('id2intent','rb') as f:
            id2intent = pickle.load(f)

        output,hidden,_ = self.encoder(query_text,query_text_len)
        query_vectors = self.att(output,output.size(1))
        res_socres = []
        for i in range(Classes_vectors.size(0)):  
            score_i = self.relation_score(query_vectors,Classes_vectors[i].repeat(query_vectors.size(0),1)).squeeze()#type,tensor
            res_socres.append(score_i)
        
        #print(type(res_socres))
        res_socres = torch.stack(res_socres,dim=0)
        #print(type(res_socres))
        return res_socres
  

In [14]:

def all_sq(intent2text,intent_texts_len,way,shot):
      
    train_classes = list(intent2text.keys())
    
    support_idx, query_idx = {},{}
    #本来是输出5个class的信息，

    for class_i in train_classes: 
        sample_set = []
        sample_set = intent2text[str(class_i)]#二维数组.intent对应的所有text
        text_content,text_lens ,sample_s_nums = [],[],[]
        
        #sample_s_nums找出intent2text某一个intent（sample_set）下面，数组选哪几个位置的text，位置集合
        sample_s_nums = np.random.permutation(len(sample_set))[:shot] 
        text_content = [sample_set[item] for item in sample_s_nums]
        text_lens = [intent_texts_len[str(class_i)][num] for num in sample_s_nums]
        support_idx[str(class_i)] = {'text_content':text_content,'text_len':text_lens}

    return support_idx